# Advanced Querying Mongo

In [1]:
!pip install pymongo

     |████████████████████████████████| 546 kB 912 kB/s eta 0:00:01


Importing libraries and setting up connection

In [3]:
#from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [6]:
db = client.get_database("ironhack")

In [7]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [9]:
db.list_collection_names()

['companies']

In [10]:
c = db.get_collection("companies")

In [11]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [68]:
c.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [26]:
# Your Code
#proyec = {"name":1, "_id":0}
#query = {"name": "Babelgum"}
#Babelgum = list(c.find(query,  proyec))
#Babelgum

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [25]:
# Your Code
#query = {"number_of_employees": {"$gt": 5000}}
#cosa = list(c.find(query).sort("number_of_employees", -1))
#cosa [0:20]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [31]:
# Your Code
#cond1 = {"founded_year": {"$gt": 1999}}
#cond2 = {"founded_year": {"$lt": 2006}}
#proyec = {"name":1, "_id":0, "founded_year":1}
#cosa = list(c.find({"$and": [cond1,cond2]},proyec))
#cosa

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [62]:
# Your Code
#cond1 = {"founded_year": {"$lt": 2010}}
#cond2 = {"ipo.valuation_amount": {"$gt": 100000000}}
#proyec = {"name":1, "_id":0, "ipo":1}
#cosa = list(c.find({"$and": [cond1,cond2]},proyec))
#cosa

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [42]:
# Your Code
#cond = {"number_of_employees": {"$lt": 1000}}
#cond2 = {"founded_year": {"$lt": 2005}}
#cosa = list(c.find(query).sort("number_of_employees", -1))
#cosa [0:10]

### 6. All the companies that don't include the `partners` field.

In [44]:
# Your Code
#cond_exi = {"partners": {"$exists": False}}
#cosa = list(c.find(cond_exi))
#cosa

### 7. All the companies that have a null type of value on the `category_code` field.

In [98]:
# Your Code
#list(c.find({"category_code": {"$in":[]}}).limit(2))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [50]:
# Your Code
#cond1 = {"number_of_employees": {"$lt": 1000}}
#cond2 = {"number_of_employees": {"$gt": 99}}
#proyec = {"name": 1, "number_of_employees": 1, "_id": 0}
#cosa = list(c.find({"$and": [cond1,cond2]},proyec))
#cosa

### 9. Order all the companies by their IPO price in a descending order.

In [60]:
# Your Code
#cond_IPO = {"ipo.valuation_amount": {"$exists": True}}
#proyec = {"name": 1, "ipo.valuation_amount": 1, "_id": 0}
#cosa = list(c.find(cond_IPO, proyec).sort("ipo.valuation_amount", -1))
#cosa

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [67]:
# Your Code
#proyec = {"name": 1, "number_of_employees": 1, "_id": 0}
#cosa = list(c.find({},proyec).sort("number_of_employees", -1))
#cosa[:11]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [74]:
# Your Code
#proy11 = {"founded_month": 1, "name": 1}
#list(c.find({"founded_month": {"$in": [7,8,9,10,11,12]}}, proy11))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [75]:
# Your Code
#proy12 = {"acquisition": 1, "name": 1, "founded_year": 1}
#cond121 = {"founded_year":{"$lt": 2000}}
#cond122 = {"acquisition.price_amount": {"$gt": 10000000}}
#list(c.find({"$and": [cond121,cond122]},proy12).sort("acquisition.price_amount",-1))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [77]:
# Your Code
#proy13 = {"acquisition": 1, "name": 1, }
#cond131 = {"acquisition.acquired_year":{"$gte": 2010}}
#list(c.find({"$and": [cond131]},proy13).sort("acquisition.price_amount",-1))

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [83]:
# Your Code
#proyec = {"name": 1, "founded_year": 1, "_id": 0}
#cosa = list(c.find({},proyec).sort("founded_year", -1))
#cosa[:11]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [85]:
# Your Code
#proy15 = {"founded_day": 1, "name": 1,"acquisition.price_amount": 1}
#cosa = list(c.find({"founded_day": {"$in": [1,2,3,4,5,6,7]}}, proy15).sort("acquisition.price_amount",-1))
#cosa[:11]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [88]:
# Your Code
#proy16 = {"number_of_employees": 1, "name": 1, "category_code": 1}
#cond161 = {"number_of_employees":{"$lte": 4000}}
#cond162 = {"category_code": "web"}
#list(c.find({"$and": [cond161,cond162]},proy16).sort("number_of_employees",1).limit(2))

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [90]:
# Your Code
#proy17 = {"acquisition": 1, "name": 1, }
#cond171 = {"acquisition.price_currency_code": "EUR"}
#cond172 = {"acquisition.price_amount": {"$gte": 10000000}}
#list(c.find({"$and": [cond171,cond172]},proy17).sort("number_of_employees",1).limit(1))

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [92]:
# Your Code
#proy18 = {"acquisition": 1, "name": 1, }
#list(c.find({"acquisition.acquired_month": {"$in": [1,2,3]}}, proy18).limit(1))

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [99]:
# Your Code
#cond1 = {"founded_year": {"$gt": 1999}}
#cond2 = {"founded_year": {"$lt": 2011}}
#cond3 = {"acquisition.acquired_year": {"$gt": 2011}}
#proyec = {"name":1, "_id":0, "acquisition.acquired_year":1}
#cosa = list(c.find({"$and": [cond1,cond2,cond3]},proyec))
#cosa

### 20. All the companies that have been 'deadpooled' after the third year.

In [101]:
# Your Code
cosa = c.find({$where: function() { return (deadpooled_year - founded_year >= 3)}})
cosa

SyntaxError: invalid syntax (<ipython-input-101-e0742cafcfcc>, line 2)